In [5]:
from importlib import import_module
from pprint import pprint

from deep_image_matching import logger, timer
from deep_image_matching.config import Config
from deep_image_matching.image_matching import ImageMatching
from deep_image_matching.io.h5_to_db import export_to_colmap

#### Define the configuration

Get the list of possible configurations and chose one of them.

In [6]:
print("Available configurations:")
pprint(Config.get_pipelines())
print("Available matching strategy:")
pprint(Config.get_matching_strategies())

Available configurations:
['superpoint+lightglue',
 'superpoint+lightglue_fast',
 'superpoint+superglue',
 'disk+lightglue',
 'aliked+lightglue',
 'orb+kornia_matcher',
 'sift+kornia_matcher',
 'loftr',
 'se2loftr',
 'roma',
 'keynetaffnethardnet+kornia_matcher',
 'dedode+kornia_matcher']
Available matching strategy:
['bruteforce',
 'sequential',
 'retrieval',
 'custom_pairs',
 'matching_lowres',
 'covisibility']


Now you have to build a dictionary with the input processing parameters (they are the same as the input parameters for the CLI and GUI) and pass it to the Config class to get the initialize the configuration object.
Refer to the README for more information about the parameters.

Note that there are two possible approaches for defining the paths needed for the processing (i.e., input images and output results):
- You can pass a single parameter defining the processing directory (with 'dir' key). Deep-Image-Matching will search for the images inside an 'image' subdirectory and will save the results in a 'results_{processing_params}' subdirectory, where {processing_params} are some information on the processing parameters used.
- or you can manually specify the input images directory (with 'images' key) and the output results directory (with 'outs' key).

Note, that these parameters are the same as the ones used in the CLI (the GUI is not updated yet).

In [7]:
cli_params = {
    "dir": "../assets/example_cyprus",
    "pipeline": "superpoint+lightglue",
    "strategy": "matching_lowres",
    "quality": "high",
    "tiling": "preselection",
    "skip_reconstruction": False,
    "force": True,
}
config = Config(cli_params)

2024-01-19 18:27:15 | [WARNING ] ../assets/example_cyprus/results_superpoint+lightglue_matching_lowres_quality_high already exists, but the '--force' option is used. Deleting the folder.


You can check the configuration object.

In [8]:
print("Config general:")
pprint(config.general)
print("Config extractor:")
pprint(config.extractor)
print("Config matcher:")
pprint(config.matcher)

Config general:
{'db_path': None,
 'geom_verification': <GeometricVerification.PYDEGENSAC: 1>,
 'gv_confidence': 0.99999,
 'gv_threshold': 4,
 'image_dir': PosixPath('../assets/example_cyprus/images'),
 'matching_strategy': 'matching_lowres',
 'min_inlier_ratio_per_pair': 0.25,
 'min_inliers_per_pair': 15,
 'min_matches_per_tile': 10,
 'output_dir': PosixPath('../assets/example_cyprus/results_superpoint+lightglue_matching_lowres_quality_high'),
 'overlap': None,
 'pair_file': PosixPath('../assets/example_cyprus/results_superpoint+lightglue_matching_lowres_quality_high/pairs.txt'),
 'quality': <Quality.HIGH: 3>,
 'retrieval': None,
 'skip_reconstruction': False,
 'tile_overlap': 10,
 'tile_preselection_size': 1000,
 'tile_selection': <TileSelection.PRESELECTION: 3>,
 'tile_size': (2400, 2000),
 'try_match_full_images': False,
 'upright': False,
 'verbose': False}
Config extractor:
{'keypoint_threshold': 0.0005,
 'max_keypoints': 4096,
 'name': 'superpoint',
 'nms_radius': 3}
Config matc

If you know what you are doing, you can update some config parameters directly updating the config dictionary (check the file config.py in the scr folder for the available parameters).


In [9]:
# - General configuration
config.general["min_inliers_per_pair"] = 10
config.general["min_inlier_ratio_per_pair"] = 0.2

# - SuperPoint configuration
config.extractor["max_keypoints"] = 8000

# - LightGue configuration
config.matcher["filter_threshold"] = 0.1

# Save configuration to a json file in the output directory
config.save()

For simplicity, save some of the configuration parameters in variables.

In [10]:
imgs_dir = config.general["image_dir"]
output_dir = config.general["output_dir"]
matching_strategy = config.general["matching_strategy"]
extractor = config.extractor["name"]
matcher = config.matcher["name"]

#### Initialize the ImageMatching class 
This will be used for performing the image matching.

In [11]:
img_matching = ImageMatching(
    imgs_dir=imgs_dir,
    output_dir=output_dir,
    matching_strategy=matching_strategy,
    local_features=extractor,
    matching_method=matcher,
    pair_file=config.general["pair_file"],
    retrieval_option=config.general["retrieval"],
    overlap=config.general["overlap"],
    existing_colmap_model=config.general["db_path"],
    custom_config=config.as_dict(),
)

Loaded SuperPoint model
Loaded SuperPoint model
2024-01-19 18:27:16 | [INFO    ] Running image matching with the following configuration:
2024-01-19 18:27:16 | [INFO    ]   Image folder: ../assets/example_cyprus/images
2024-01-19 18:27:16 | [INFO    ]   Output folder: ../assets/example_cyprus/results_superpoint+lightglue_matching_lowres_quality_high
2024-01-19 18:27:16 | [INFO    ]   Number of images: 10
2024-01-19 18:27:16 | [INFO    ]   Matching strategy: matching_lowres
2024-01-19 18:27:16 | [INFO    ]   Image quality: Quality.HIGH
2024-01-19 18:27:16 | [INFO    ]   Tile selection: TileSelection.PRESELECTION
2024-01-19 18:27:16 | [INFO    ]   Retrieval option: None
2024-01-19 18:27:16 | [INFO    ]   Overlap: None
2024-01-19 18:27:16 | [INFO    ]   Feature extraction method: superpoint
2024-01-19 18:27:16 | [INFO    ]   Matching method: lightglue


#### Generate pairs to be matched

In [12]:
pair_path = img_matching.generate_pairs()
timer.update("generate_pairs")

2024-01-19 18:27:16 | [INFO    ] Low resolution matching, generating pairs ..
Loaded SuperPoint model
2024-01-19 18:27:16 | [INFO    ] Extracting features from downsampled images...


100%|██████████| 10/10 [00:00<00:00, 12.94it/s]

2024-01-19 18:27:17 | [INFO    ] Matching downsampled images...



100%|██████████| 45/45 [00:01<00:00, 38.85it/s]

2024-01-19 18:27:18 | [INFO    ] Found 28 pairs.


Try to rotate images so they will be all "upright", useful for deep-learning approaches that usually are not rotation invariant

In [13]:
if config.general["upright"]:
    img_matching.rotate_upright_images()
    timer.update("rotate_upright_images")

#### Extract features

In [14]:
feature_path = img_matching.extract_features()
timer.update("extract_features")

2024-01-19 18:27:18 | [INFO    ] Extracting features with superpoint...
2024-01-19 18:27:18 | [INFO    ] superpoint configuration: 
{'keypoint_threshold': 0.0005,
 'max_keypoints': 8000,
 'name': 'superpoint',
 'nms_radius': 3}


100%|██████████| 10/10 [00:03<00:00,  3.20it/s]

2024-01-19 18:27:21 | [INFO    ] Features extracted!


#### Run matching

In [15]:
match_path = img_matching.match_pairs(feature_path)
timer.update("matching")

2024-01-19 18:27:21 | [INFO    ] Matching features with lightglue...
2024-01-19 18:27:21 | [INFO    ] lightglue configuration: 
{'depth_confidence': 0.95,
 'filter_threshold': 0.1,
 'flash': True,
 'mp': False,
 'n_layers': 9,
 'name': 'lightglue',
 'width_confidence': 0.99}
2024-01-19 18:27:21 | [INFO    ] Matching features...
2024-01-19 18:27:21 | [INFO    ] 


100%|██████████| 28/28 [00:22<00:00,  1.22it/s]


If features have been extracted on "upright" images, this function bring features back to their original image orientation

In [16]:
if config.general["upright"]:
    img_matching.rotate_back_features(feature_path)
    timer.update("rotate_back_features")

#### Export in colmap format

In [17]:
database_path = output_dir / "database.db"
export_to_colmap(
    img_dir=imgs_dir,
    feature_path=feature_path,
    match_path=match_path,
    database_path=database_path,
    camera_model="simple-radial",
    single_camera=True,
)
timer.update("export_to_colmap")

100%|██████████| 10/10 [00:00<00:00, 808.15it/s]
28it [00:00, 5240.77it/s]             


#### Run reconstruction
If --skip_reconstruction is not specified (from CLI or in the cli_params dictonary), run reconstruction with pycolmap

Try first to import the pycolmap module, if it fails, skip reconstruction



In [18]:
if not config.general["skip_reconstruction"]:
    use_pycolmap = True
    try:
        pycolmap = import_module("pycolmap")
    except ImportError:
        logger.error("Pycomlap is not available.")
        use_pycolmap = False

If the pycolmap module is imported, define some parameters for the reconstruction and run it.


- Define the pycolmap database path (default is output_dir / "database_pycolmap.db")
- 
- Define the pycolmp CameraMode, that is how pycolmap create the cameras.

  Possible CameraMode are:
    - CameraMode.AUTO: infer the camera model based on the image exif
    - CameraMode.PER_FOLDER: create a camera for each folder in the image directory
    - CameraMode.PER_IMAGE: create a camera for each image in the image directory
    - CameraMode.SINGLE: create a single camera for all images
    - 
- (Optional) You can manually define the cameras parameters. Otherwise, set it to `None`.
  Refer to [https://github.com/colmap/colmap/blob/main/src/colmap/sensor/models.h](https://github.com/colmap/colmap/blob/main/src/colmap/sensor/models.h).    
    - OPENCV camera models and number of parameters to be used
    - SIMPLE_PINHOLE: f, cx, cy
    - PINHOLE: fx, fy, cx, cy
    - SIMPLE_RADIAL: f, cx, cy, k
    - RADIAL: f, cx, cy, k1, k2
    - OPENCV: fx, fy, cx, cy, k1, k2, p1, p2
    - OPENCV_FISHEYE: fx, fy, cx, cy, k1, k2, k3, k4
    - FULL_OPENCV: fx, fy, cx, cy, k1, k2, p1, p2, k3, k4, k5, k6
    - FOV: fx, fy, cx, cy, omega
    - SIMPLE_RADIAL_FISHEYE: f, cx, cy, k
    - RADIAL_FISHEYE: f, cx, cy, k1, k2
    - THIN_PRISM_FISHEYE: fx, fy, cx, cy, k1, k2, p1, p2, k3, k4, sx1, sy1
  
  **Note 1**: the cameras are first detected with the CameraMode specified and then overwitten with the custom model. Therefore, you MUST provide the SAME NUMBER of cameras and with the SAME ORDER in which the cameras appear in the COLMAP database. To see the camera number and order, you can run the reconstruction a first time with the AUTO camera mode (and without manually define the cameras) and see the list of cameras in the database with
  ```python print(list(model.cameras.values()))``` 
  or opening the database with the COLMAP gui.      

  ```python
   cam0 = pycolmap.Camera(
      model="PINHOLE",
      width=1500,
      height=1000,
      params=[1500, 1500, 750, 500],
   )
   cam1 = pycolmap.Camera(
       model="SIMPLE_PINHOLE",
       width=6012,
       height=4008,
       params=[9.267, 3.053, 1.948],
   )
   cameras = [cam0, cam1]
  ```
  or 
  ```python
  cameras = None
  ```

  **Note2**: Use the SIMPLE-PINHOLE camera model if you want to export the solution to Metashape, as there are some bugs in COLMAP (or pycolamp) when exporting the solution in the Bundler format.
  e.g., using FULL-OPENCV camera model, the principal point is not exported correctly and the tie points are wrong in Metashape.

- (Optional) You can specify some reconstruction configuration in a dictionary, or leave the dictionary empty to use the default configuration.

  ``` python  
  reconst_opts = {
          "ba_refine_focal_length": True,
          "ba_refine_principal_point": False,
          "ba_refine_extra_params": False,
      }
  ```
  or
  ``` python
  reconst_opts = {}
  ```



In [19]:
if not config.general["skip_reconstruction"] and use_pycolmap:
    # import reconstruction module
    from deep_image_matching import reconstruction

    # Define database path and camera mode
    database = output_dir / "database_pycolmap.db"
    camera_mode = pycolmap.CameraMode.AUTO
    cameras = None
    reconst_opts = {}
    model = reconstruction.main(
        database=database,
        image_dir=imgs_dir,
        feature_path=feature_path,
        match_path=match_path,
        pair_path=pair_path,
        sfm_dir=output_dir,
        camera_mode=camera_mode,
        cameras=cameras,
        skip_geometric_verification=True,
        reconst_opts=reconst_opts,
        verbose=config.general["verbose"],
    )

    timer.update("pycolmap reconstruction")


2024-01-19 18:27:44 | [INFO    ] Creating an empty database...
2024-01-19 18:27:44 | [INFO    ] Importing images into the database...
2024-01-19 18:27:44 | [INFO    ] Importing features into the database...


100%|██████████| 10/10 [00:00<00:00, 1696.79it/s]

2024-01-19 18:27:44 | [INFO    ] Importing matches into the database...



28it [00:00, 8820.83it/s]             

2024-01-19 18:27:44 | [INFO    ] Running 3D reconstruction...



I20240119 18:27:44.792718 312187 misc.cc:198] 
Loading database
I20240119 18:27:44.793233 312187 database_cache.cc:54] Loading cameras...
I20240119 18:27:44.793254 312187 database_cache.cc:64]  2 in 0.000s
I20240119 18:27:44.793262 312187 database_cache.cc:72] Loading matches...
I20240119 18:27:44.793620 312187 database_cache.cc:78]  28 in 0.000s
I20240119 18:27:44.793637 312187 database_cache.cc:94] Loading images...
I20240119 18:27:44.794569 312187 database_cache.cc:143]  10 in 0.001s (connected 8)
I20240119 18:27:44.794591 312187 database_cache.cc:154] Building correspondence graph...
I20240119 18:27:44.799369 312187 database_cache.cc:190]  in 0.005s (ignored 0)
I20240119 18:27:44.799384 312187 timer.cc:91] Elapsed time: 0.000 [minutes]
I20240119 18:27:44.799769 312187 misc.cc:198] 
Finding good initial image pair
I20240119 18:27:44.968331 312187 misc.cc:198] 
Initializing with image pair #5 and #9
I20240119 18:27:44.974197 312187 misc.cc:198] 
Global bundle adjustment
I20240119 18

2024-01-19 18:27:49 | [INFO    ] Reconstructed 1 model(s).
2024-01-19 18:27:49 | [INFO    ] Largest model is #0 with 8 images.
2024-01-19 18:27:49 | [INFO    ] Reconstruction statistics:
Reconstruction:
	num_reg_images = 8
	num_cameras = 1
	num_points3D = 8344
	num_observations = 39058
	mean_track_length = 4.68097
	mean_observations_per_image = 4882.25
	mean_reprojection_error = 1.06744
	num_input_images = 10


I20240119 18:27:49.365422 312187 timer.cc:91] Elapsed time: 0.076 [minutes]


#### Import the solution in Metashape

If the reconstruction with pycolmap has been performed, you can import the solution in Metashape.
This is done by first exporting the solution in Bundler format and then importing it in Metashape.

This can be performed automatically with the function `export_to_metashape()`, which can also run a Bundle Adjustment.

**Note** that this function is under development and it is not yet integrated in Deep-Image-Matching (but it is inside a script in the scripts directory). However, you can use it as an example to export the solution to Metashape.

In [20]:
# import the function from the script folder
from scripts.metashape.metashape_from_dim import export_to_metashape


# Define the paths for the ne Metashape project and the path of the Bundler filter
project_dir = config.general["output_dir"] / "metashape"
project_name = config.general["output_dir"].name + ".psx"
project_path = project_dir / project_name

rec_dir = config.general["output_dir"] / "reconstruction"
bundler_file_path = rec_dir / "bundler.out"
bundler_im_list = rec_dir / "bundler_list.txt"


# Define the interior orientation parameters to refine or fix during the bundle adjustment (the parameters are the same as in the Metashape GUI)

prm_to_optimize = {
    "f": True,
    "cx": True,
    "cy": True,
    "k1": True,
    "k2": True,
    "k3": True,
    "k4": False,
    "p1": True,
    "p2": True,
    "b1": False,
    "b2": False,
    "tiepoint_covariance": True,
}

# Export the reconstruction to Metashape
export_to_metashape(
    project_path=project_path,
    images_dir=config.general["image_dir"],
    bundler_file_path=bundler_file_path.resolve(),
    bundler_im_list=bundler_im_list.resolve(),
    prm_to_optimize=prm_to_optimize,
)

ModuleNotFoundError: No module named 'scripts'

In [ ]:
timer.print()